<a href="https://colab.research.google.com/github/charris561/Discrete-Structures/blob/main/Projects/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 2150 Computer Project 2: Fun with RSA

**Goals:** Experiment with cryptography in the form of RSA, while gaining experience in Python and Jupyter.

**Instructions:** Carefully read all of the text in the Markdown cells; these give you information about the assignment. **Numbered, heading-level text at the bottom of the document describes the project deliverables.** Cells requesting the implementation of a function also include `assert` statements. Use these statements as a guide to the expected output of your function; if the assertions fail, then the function is invalid.

**Note: Include each deliverable directly underneath the markdown cell which asks for it.**

**Note also: when we receive your notebook, the first thing we will do is clear its memory and run it from the beginning.** You are responsible for ensuring that when we do this, the entire notebook executes and computes the desired results. Notebooks which fail to execute will typically receive low grades.

**Note this too: hard-coded answers, even if correct, will be awarded few if any points.** Your answers should be generated programmatically, not derived offline and then hard-coded.

## Introduction

Modern public key cryptography uses a pair of keys to protect sensitive data. One key is made public and the other is kept secrete by its owner. Persons wishing to communicate securely with the owner of a key pair will encrypt messages using the public key, and only the owner can decrypt those messages with the private key. Another use case is digital signatures, where the owner will encrypt metadata about the message using the private key. Anyone with the public key can decrypt such a signature that only the private key could have created.

Central to the operation of this system is what is called *modular arithmetic.* We won't get into the specifics, but if you are interested the book has a good introduction. For our purposes, we just need to know that when taking the *modulus* of one number with another, we are looking for the *remainder* we would find when dividing one with the other. So, for example, $42\,mod\,17 = 8$, because $2\times 17 + 8 = 42$.

We can use this operator to define even and odd: a number $n$ is *even* if $n\,mod\,2=0$, and a number is odd if $n\,mod\,2=1$.

## Greatest Common Divisor
Another application of modular arithmetic is in [Euclid's Algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm) for finding the Greatest Common Divisor (GCD) of two numbers. The algorithm performs a recursive operation starting with the two numbers, $a$ and $b$ for which the GCD is desired. The first step is to calculate $r=a\,mod\,b$. If $r=0$, then $b$ is the GCD. If $r\neq 0$, the algorithm returns the result of $GCD(b,r)$.

In [1]:
def GCD(a, b):
    r = a % b
    if r == 0: return b
    return GCD(b, r)

[GCD(72,7),GCD(24,30)]

[1, 6]

## Extended Euclidean Algorithm
There is another version of Euclid's algorithm known as the [Extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) which keeps track of the quotient of $a/b$ as well as two additional integers $s$ and $t$ of [Bézout's identity](https://en.wikipedia.org/wiki/B%C3%A9zout%27s_identity), which we will put to good use later.

In the Extended Euclidean Algorithm, we keep track of three sequences of numbers defined as follows:
\begin{eqnarray*}
r_0 = a \quad& r_1 = b &\quad r_i = r_{i-2}-q_ir_{i-1} \\
s_0 = 1 \quad& s_1 = 0 &\quad s_i = s_{i-2}-q_is_{i-1} \\
t_0 = 0 \quad& t_1 = 1 &\quad t_i = t_{i-2}-q_it_{i-1} \\
\end{eqnarray*}
As in the earlier algorithm, the extended algorithm works recursively until it finds some $r_i=0$ at which point it is done. First, we check the modulus $r$ of $a$ and $b$. If $r$ is not $0$, we proceed to finding the current value of $q$ which is the integer quotient of $a$ and $b$. We then use $q$ to find the next values of $s$ and $t$. We then repeat the process until we encounter an $r=0$.

#### Example 1: EGCD(30,24)
| i | r  | s  | t  | q |
|---|----|----|----|---|
| 0 | 30 | 1  | 0  |   |
| 1 | 24 | 0  | 1  |   |
| 2 | 6  | 1  | -1 | 1 |
| 3 | 0  |    |    |   |

#### Example 2: EGCD(72,7)
| i | r  | s  | t   | q  |
|---|----|----|-----|----|
| 0 | 72 | 1  | 0   |    |
| 1 | 7  | 0  | 1   |    |
| 2 | 2  | 1  | -10 | 10 |
| 3 | 1  | -3 | 31  | 3  |
| 4 | 0  |    |     |    |

At this point, where $r_i=0$, then $r_{i-1}$ is the GCD, and we'll have a keen interest in the values in that row.

# Deliverable #1: A function implementing the Extended Euclidean Algorithm
The function should take arguments `a` and `b` an return the values of $r_{i-1}$, $s_{i-1}$, $t_{i-1}$, and $q_{i-1}$ where $r_i=0$. Two assertions are provided to check your work.

In [2]:
import math

def egcd(a,b):
    # TODO: implement Extended Euclidean Algorithm and return the values r,s,t, and q from the row *before* r = 0

    #define r, s, t, and q arrays
    rr = [a, b]
    ss = [1, 0]
    tt = [0, 1]
    qq = []

    #loop until r_i = 0
    i = 2
    r = rr[i-2] % rr[i-1]
    while (r != 0):
      r = rr[i-2] % rr[i-1]
      rr.append(r)
      q = rr[i-2] // rr[i-1]
      qq.append(q)
      s = ss[i-2] - q * ss[i-1]
      ss.append(s)
      t = tt[i-2] - q * tt[i-1]
      tt.append(t)
      i = i+1

    return [rr[-2],ss[-2],tt[-2],qq[-2]]
    
assert egcd(72,7) == [1,-3, 31,3]
assert egcd(24,30) == [6, -1, 1, 1]

In [3]:
egcd(35, 30)

[5, 1, -1, 1]

Why all this extra work? Well, Bézout's identity can be given by the equation: <mark>$as_j+bt_j=GCD(a,b)$ where $j=i-1$</mark>, which will come in handy in a few moments.

## Rivest, Shamir, and Adleman
In 1977, Ron Rivest, Adi Shamir, and Leonard Adleman described a very strong public key cryptography system. The key (pun intended) to this form of encryption is to find three numbers <mark>$e$, $d$, and $n$</mark> such that when given a message <mark>$M$: $M^{e^d}\,mod\,n = M$</mark>. The "owner" of this set of such integers can <mark>share $e$ and $n$</mark> so that others can encrypt messages that only she can read. Traditionally, such a person is refered to as Alice, and the person to whom she shares this "public" information is called Bob.

So, Alice gives Bob <u>$e$ and $n$</u>. Bob has an important message that only Alice should see, so by some previously agreed means, he converts the <u>message into a series of numbers, $M$</u>. For each <mark>$m\in M$ he calculates $c = m^e\,mod\,n$ and assembles $C = [c_0,\cdots c_n]$ and sends $C$ to Alice</mark>. Alice then uses <u>$d$</u>, which she has kept to herself in all the world, to calculate from each <mark>$c\in C$, $m = c^d\,mod\,n$. She then assembles $M=[m_0\cdots m_n]$ and reversing the previously agreed conversion from message to numbers, retrieves the secret message.</mark>

Sounds amazing, right? So, just how do we find such a nifty set of <mark>$e$, $d$, and $n$</mark>? Here's the recipe:

1. Select <b><u>two large primes, $p$ and $q$ </b></u>and calculate <b><u>$n=p\times q$</b></u>. We want really large numbers, since we want factoring <b><u>$n$ </b></u>to be computationally difficult (think 2^{2048} and you're in the right ballpark).
2. Calculate <b><u>$k=(p-1)(q-1)$ </b></u>and select some prime<b><u> $e$ such that $1<e<k$ and $e$ is not a factor of $k$</b></u>.
3. Using <b><u>$e$ and $k$, find $d$ such that $de-vk=1$. Here $k\to a$, $e\to b$, and $d$ is the output of our function above</b></u>.

Oh my. <b><u>$de-vk=1$</b></u>? Well, we know <b><u>$e$</b></u>, since we picked it. And we know <b><u>$k$</b></u>, since that came from our choice of <b><u>$p$ and $q$</b></u>. So, if we let <b><u>$a=k$ and $b=e$, then $gcd(a,b) = gcd(k,e) = 1$ since $e$ is prime</b></u>. And we have an equation that might look eerily familiar: <b><u>$db-va=1$</b></u>.

No? What if we let <b><u>$-v=s_j$, $d=t_j$</b></u>, and rearranged a little: <b><u>$as_j+bt_j=GCD(a,b)$</b></u>?

### Ah ha!

We know <b><u>$e$, we know $n$, and we want $d$</b></u>. So substituting back in, we see that we want to find <b><u>$t_j$</b></u>. So, it would be handy to have a nice method to give us that directly out of the results of our extended Euclidean Algorithm: 

In [4]:
def find_d(k,e):
    return egcd(k,e)[2]

assert find_d(72,7) == 31
assert find_d(1449000,7907) == 643043

## Deliverable #2: $n$, $e$, and $d$

Now that we have the tools, let's generate a key pair! Using $p=1381$, $q=1051$ and $e=7907$, create a public/private key set. The key set should return `[n,e,d]`.

In [5]:
def keyset(p, q, e):
    # TODO: calculate and return n, e, and d

    #step one: calculate n
    n = p * q

    #step two: calculate k 
    k = (p - 1) * (q - 1)

    #step three: find d
    d = find_d(k, e)

    print ([n, e, d])

    return [n, e, d]

[n, e, d] = keyset(1381,1051,7907)
assert [n, e, d] == [1451431, 7907, 643043] 

[1451431, 7907, 643043]


## Deliverable #3: Encrypt Something
An important part of any digital communication is  choice of encoding, that is "how does one convert from letters and punctuation to numbers?" For this project, we'll simply use Unicode employing Python's `ord` function to convert from a character to an integer. For this deliverable, create a function `encrypt` that when given a string, $e$, and $n$ returns an array of integers corresponding to the characters in the string encrypted using $c=m^e\,mod\,n$. 

**Python Tip:** When calculating $m^e\,mod\,n$, use the `pow` function instead of the `**` and `%` operators.

**Python Tip:** You can *iterate* over the characters in a string just like you might with a list! For example: `[letter for letter in "word"]` will give `['w','o','r','d']`.

In [6]:
M = 'hello there secret friend'
def encrypt(message, e, n):
    # TODO: encrypt the message character by character

    #iterate over string and convert characters to unicode using the ord function as described above
    C = []
    for letter in M:
        m = ord(letter)
        C.append(pow(m, e, n)) 

    return C

cipher = encrypt(M, e, n)
print(cipher)
assert cipher == [1041244, 739369, 892978, 892978, 799576, 304346, 1398703, 1041244, 739369, 38960, 739369, 304346, 642935, 739369, 1079616, 38960, 739369, 1398703, 304346, 360690, 38960, 722667, 739369, 462214, 282605]

[1041244, 739369, 892978, 892978, 799576, 304346, 1398703, 1041244, 739369, 38960, 739369, 304346, 642935, 739369, 1079616, 38960, 739369, 1398703, 304346, 360690, 38960, 722667, 739369, 462214, 282605]


## Deliverable #4: Decrypt Something
Now create a function that when given an array of integers produced by the `encrypt` function, returns the original message.

**Python Tip:** To convert from the integer output of `ord`, use `chr`.

**Python Tip:** To convert from an array of characters to a string, try `"".join(theArray)`.

**Python Tip:** To concatenate strings `'abc'` and `'def'`, try `'abc' + 'def'`

**Python Tip:** To raise a big number `a` to a large power `b` and then return the result mod `c`, use `pow(a,b,c)`. This is much faster than `(a**b)%c`.

In [7]:
def decrypt(cipher, d, n):
    # TODO: implement decrypt + decode

    #converts letters in cipher to coded message
    decryptedCipher = []
    for letter in cipher:
        decryptedCipher.append(pow(letter, d, n))
    
    #converts letters in cipher back to string
    decrypted_message = ""
    for letter in decryptedCipher:
        currentChr = chr(letter)
        decrypted_message += currentChr

    print (decrypted_message)

    return decrypted_message

message = decrypt(cipher, d, n)
assert message == M

hello there secret friend


## Breaking RSA
One way to break our encryption is to uncover what the original values $p$ and $q$ were. With production ready systems, which employ *very* large values for $p$ and $q$, this becomes computationally very expensive. Look forward to CS4720 to appreciate what that really means. But, for now we've selected some relatively small values that aren't terribly expensive to factor.

A simple algorithm for factoring for our purposes is to just check all the numbers less than the square root of the number we're trying to factor. For each number, $p_1$, we find that evenly divides $n$, we have a companion $q_1=n/p_1$. Since we know that $p$ and $q$ selected for RSA are prime, we know that only $p$ and $q$ will evenly divide $n$, so we just need to find that first $p$.

## Deliverable #5: Naïve Factoring
Create a function that returns the prime factors of the argument $n$.

In [8]:
def naivefactors(n):
    # TODO: implement a function that factors a semi-prime into an array of primes
    # note: you may assume that n=p*q where p>1 and q>1

    #we need to check every value under the square root of n
    upperBound = int(math.sqrt(n))
    lowerBound = 1
    a = []
    for p1 in range(lowerBound, upperBound):

        #from here, if p1 evenly divides n we can compute q1 = n/p1
        if ((n) % p1 == 0):
            q1 = n/p1
            a = [p1, q1]

    return a

assert naivefactors(21) == [3,7]
assert naivefactors(59444051) == [7703,7717]

Pretend your name is Eve. You intercepted a message Bob sent to Alice. You know the values $n$ and $e$ of Alice's public key. (Note: these are different than above):

In [9]:
[n, e] = [39217303, 7687]
C = [7473306,34860190,31360573,20968028,9070305,20827012,34743153,11419633,36622909,20827012,34743153,7882764,34860190,31360573,21566064,7163950,34860190,31360573,7163950,13366249,34860190,11419633,9070305,7163950,20827012,19210583,29100039,7882764,131312,12921995,131312,21521610,131312,31360573,34743153,4735271,7163950,9665260,7882764,34743153,29100039,7163950,131312,12921995,131312,9070305,13366249,7163950,20827012,19210583,19210583,34860190,21521610,10677701,36622909,7882764,21566064,29100039,21521610,131312,31360573,34743153,7467220,7163950,34743153,29100039,131312,9070305,131312,7163950,7882764,21566064,7163950,20827012,31360573,7163950,131312,31360573,18965078,4735271,7163950,9665260,7882764,34743153,29100039,7163950,131312,12921995,131312,9070305,13366249,7163950,131312,31360573,18965078,7163950,34743153,29100039,131312,9070305,131312,7163950,7882764,21566064,7163950,20827012,36622909,15592578,20827012,13366249,21566064,7163950,20827012,7163950,31360573,131312,15592578,7163950,8995063,131312,20968028,7882764,31360573,31360573,7882764,31360573,20968028,4735271,7163950,32047640,7882764,33864881,131312,7163950,19210583,34860190,31360573,34743153,7882764,31360573,11419633,20827012,36622909,36622909,13366249,7163950,21521610,34860190,12921995,131312,21566064,7163950,33864881,9070305,34860190,21521610,7163950,20827012,7163950,8995063,131312,20968028,7882764,31360573,31360573,7882764,31360573,20968028,4735271,4735271,4735271,7163950,34743153,34860190,7163950,20827012,31360573,7163950,131312,31360573,18965078,4735271,4735271,4735271,34743153,34860190,7163950,20827012,7163950,31360573,131312,15592578,7163950,8995063,131312,20968028,7882764,31360573,31360573,7882764,31360573,20968028,4735271]

## Deliverable #6: Uncover the secret message
Your mission:
1. Use your `naivefactors` function to determine $p$ and $q$,
2. Use your `keyset` function to generate Alice's complete key set,
3. Use Alice's key set to decrypt Bob's message.

In [10]:
# TODO: complete the mission
p = int(naivefactors(n)[0])
q = int(naivefactors(n)[1])
[n, e, d] = keyset(p, q, e)
decryptedMessage = decrypt(C, d, n)


[39217303, 7687, 18651151]
Congratulations on your achievement. With every accomplishment, there is an end. With every end there is always a new beginning. Life continually moves from a beginning... to an end...to a new beginning.
